In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

  Using cached protobuf-3.19.6-cp39-cp39-macosx_10_9_x86_64.whl (980 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)


  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 20210226132247
    Uninstalling flatbuffers-20210226132247:
      Successfully uninstalled flatbuffers-20210226132247
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1


In [2]:
!pip install keras-image-helper

In [8]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [9]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [12]:
from keras_image_helper import create_preprocessor

In [13]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [16]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [17]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [18]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [20]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [21]:
preds = pb_response.outputs['dense_7'].float_val

In [22]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [23]:
dict(zip(classes, preds))

{'dress': -1.8682901859283447,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.88715934753418,
 'shirt': -2.8124334812164307,
 'shoes': -3.666282892227173,
 'shorts': 3.200361490249634,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835045337677002}